# 1. Cleaning

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.float_format", "{:.4f}".format)

DATA_PATH = Path("home-credit-default-risk")

# -----------------------
# 1) LOAD DATA
# -----------------------
def load_home_credit_data(data_path: Path):
    files = {
        "application_train": "application_train.csv",
        "bureau": "bureau.csv",
        "bureau_balance": "bureau_balance.csv",
        "credit_card_balance": "credit_card_balance.csv",
        "installments_payments": "installments_payments.csv",
        "POS_CASH_balance": "POS_CASH_balance.csv",
        "previous_application": "previous_application.csv",
    }
    dfs = {k: pd.read_csv(data_path / v) for k, v in files.items()}
    return dfs

# -----------------------
# 2) HELPERS
# -----------------------
def add_missing_flags(df, cols, suffix="_MISSING"):
    for c in cols:
        if c in df.columns:
            df[c + suffix] = df[c].isna().astype(np.int8)
    return df

def fix_application_anomalies(df):
    """
    Known Home Credit anomalies:
    - DAYS_EMPLOYED == 365243 is a sentinel (unknown). Replace with NaN + flag.
    - DAYS_BIRTH is negative days; create AGE_YEARS.
    """
    df = df.copy()

    if "DAYS_EMPLOYED" in df.columns:
        df["DAYS_EMPLOYED_ANOM"] = (df["DAYS_EMPLOYED"] == 365243).astype(np.int8)
        df.loc[df["DAYS_EMPLOYED"] == 365243, "DAYS_EMPLOYED"] = np.nan

    if "DAYS_BIRTH" in df.columns:
        df["AGE_YEARS"] = (-df["DAYS_BIRTH"] / 365.25).astype(float)

    # Some people also turn DAYS_EMPLOYED into years
    if "DAYS_EMPLOYED" in df.columns:
        df["EMPLOYED_YEARS"] = (-df["DAYS_EMPLOYED"] / 365.25).astype(float)

    return df

def safe_ratio(a, b):
    """Avoid division by zero."""
    return np.where((b == 0) | pd.isna(b), np.nan, a / b)

# -----------------------
# 3) AGGREGATIONS (SK_ID_CURR LEVEL)
# -----------------------
def aggregate_bureau(bureau: pd.DataFrame):
    b = bureau.copy()

    # simple flags
    b["CREDIT_ACTIVE_IS_ACTIVE"] = (b["CREDIT_ACTIVE"] == "Active").astype(int)
    b["CREDIT_ACTIVE_IS_CLOSED"] = (b["CREDIT_ACTIVE"] == "Closed").astype(int)

    agg = b.groupby("SK_ID_CURR").agg(
        bureau_loan_count=("SK_ID_BUREAU", "count"),
        bureau_active_loans=("CREDIT_ACTIVE_IS_ACTIVE", "sum"),
        bureau_closed_loans=("CREDIT_ACTIVE_IS_CLOSED", "sum"),
        bureau_total_credit_sum=("AMT_CREDIT_SUM", "sum"),
        bureau_total_credit_mean=("AMT_CREDIT_SUM", "mean"),
        bureau_debt_sum=("AMT_CREDIT_SUM_DEBT", "sum"),
        bureau_debt_mean=("AMT_CREDIT_SUM_DEBT", "mean"),
        bureau_overdue_sum=("AMT_CREDIT_SUM_OVERDUE", "sum"),
        bureau_overdue_mean=("AMT_CREDIT_SUM_OVERDUE", "mean"),
        bureau_days_credit_mean=("DAYS_CREDIT", "mean"),
        bureau_days_enddate_mean=("DAYS_CREDIT_ENDDATE", "mean"),
    ).reset_index()

    return agg

def aggregate_bureau_balance(bureau_balance: pd.DataFrame, bureau: pd.DataFrame):
    """
    bureau_balance is keyed by SK_ID_BUREAU, not SK_ID_CURR.
    So: aggregate bureau_balance per SK_ID_BUREAU -> merge to bureau -> aggregate to SK_ID_CURR.
    """
    bb = bureau_balance.copy()

    # status distributions (0,1,2,3,4,5,C,X)
    # We'll compute counts + ratios for "bad" statuses
    bad_status = {"1", "2", "3", "4", "5"}
    bb["BB_IS_BAD"] = bb["STATUS"].isin(bad_status).astype(int)

    bb_agg_bureau = bb.groupby("SK_ID_BUREAU").agg(
        bb_months_count=("MONTHS_BALANCE", "count"),
        bb_bad_months=("BB_IS_BAD", "sum"),
        bb_latest_month=("MONTHS_BALANCE", "max"),
        bb_oldest_month=("MONTHS_BALANCE", "min"),
    ).reset_index()

    # merge to bureau to get SK_ID_CURR
    b = bureau[["SK_ID_BUREAU", "SK_ID_CURR"]].merge(bb_agg_bureau, on="SK_ID_BUREAU", how="left")

    # aggregate to customer
    agg = b.groupby("SK_ID_CURR").agg(
        bb_total_months=("bb_months_count", "sum"),
        bb_total_bad_months=("bb_bad_months", "sum"),
        bb_avg_latest_month=("bb_latest_month", "mean"),
        bb_avg_oldest_month=("bb_oldest_month", "mean"),
    ).reset_index()

    # add ratio
    agg["bb_bad_ratio"] = safe_ratio(agg["bb_total_bad_months"], agg["bb_total_months"])

    return agg

def aggregate_previous_application(prev: pd.DataFrame):
    p = prev.copy()

    # contract status flags
    p["PREV_APPROVED"] = (p["NAME_CONTRACT_STATUS"] == "Approved").astype(int)
    p["PREV_REFUSED"] = (p["NAME_CONTRACT_STATUS"] == "Refused").astype(int)

    # time features: DAYS_DECISION (negative days)
    agg = p.groupby("SK_ID_CURR").agg(
        prev_app_count=("SK_ID_PREV", "count"),
        prev_approved=("PREV_APPROVED", "sum"),
        prev_refused=("PREV_REFUSED", "sum"),
        prev_credit_mean=("AMT_CREDIT", "mean"),
        prev_credit_sum=("AMT_CREDIT", "sum"),
        prev_annuity_mean=("AMT_ANNUITY", "mean"),
        prev_goods_price_mean=("AMT_GOODS_PRICE", "mean"),
        prev_days_decision_mean=("DAYS_DECISION", "mean"),
        prev_days_decision_min=("DAYS_DECISION", "min"),
    ).reset_index()

    agg["prev_approval_rate"] = safe_ratio(agg["prev_approved"], agg["prev_app_count"])
    return agg

def aggregate_installments(inst: pd.DataFrame):
    i = inst.copy()

    # Payment behavior
    i["PAYMENT_DIFF"] = i["AMT_PAYMENT"] - i["AMT_INSTALMENT"]
    i["LATE"] = (i["DAYS_ENTRY_PAYMENT"] > i["DAYS_INSTALMENT"]).astype(int)
    i["DAYS_LATE"] = (i["DAYS_ENTRY_PAYMENT"] - i["DAYS_INSTALMENT"]).clip(lower=0)

    agg = i.groupby("SK_ID_CURR").agg(
        inst_count=("SK_ID_PREV", "count"),
        inst_late_ratio=("LATE", "mean"),
        inst_days_late_mean=("DAYS_LATE", "mean"),
        inst_days_late_max=("DAYS_LATE", "max"),
        inst_payment_diff_mean=("PAYMENT_DIFF", "mean"),
        inst_payment_mean=("AMT_PAYMENT", "mean"),
        inst_instalment_mean=("AMT_INSTALMENT", "mean"),
        inst_payment_sum=("AMT_PAYMENT", "sum"),
        inst_instalment_sum=("AMT_INSTALMENT", "sum"),
    ).reset_index()

    agg["inst_paid_ratio_sum"] = safe_ratio(agg["inst_payment_sum"], agg["inst_instalment_sum"])
    return agg

def aggregate_pos_cash(pos: pd.DataFrame):
    p = pos.copy()

    p["POS_LATE"] = (p["SK_DPD"] > 0).astype(int)

    agg = p.groupby("SK_ID_CURR").agg(
        pos_records=("SK_ID_PREV", "count"),
        pos_unique_prev=("SK_ID_PREV", "nunique"),
        pos_late_ratio=("POS_LATE", "mean"),
        pos_skdpd_mean=("SK_DPD", "mean"),
        pos_skdpd_max=("SK_DPD", "max"),
        pos_months_balance_min=("MONTHS_BALANCE", "min"),
        pos_months_balance_max=("MONTHS_BALANCE", "max"),
    ).reset_index()
    return agg

def aggregate_credit_card(cc: pd.DataFrame):
    c = cc.copy()

    # utilization proxy: balance / limit (limit can be 0 or missing)
    if "AMT_CREDIT_LIMIT_ACTUAL" in c.columns and "AMT_BALANCE" in c.columns:
        c["CC_UTIL"] = safe_ratio(c["AMT_BALANCE"], c["AMT_CREDIT_LIMIT_ACTUAL"])
    else:
        c["CC_UTIL"] = np.nan

    agg = c.groupby("SK_ID_CURR").agg(
        cc_records=("SK_ID_PREV", "count"),
        cc_unique_prev=("SK_ID_PREV", "nunique"),
        cc_balance_mean=("AMT_BALANCE", "mean"),
        cc_balance_max=("AMT_BALANCE", "max"),
        cc_limit_mean=("AMT_CREDIT_LIMIT_ACTUAL", "mean"),
        cc_util_mean=("CC_UTIL", "mean"),
        cc_skdpd_mean=("SK_DPD", "mean") if "SK_DPD" in c.columns else ("CC_UTIL", "mean"),
        cc_skdpd_max=("SK_DPD", "max") if "SK_DPD" in c.columns else ("CC_UTIL", "mean"),
    ).reset_index()

    return agg

# -----------------------
# 4) MERGE ALL FEATURES
# -----------------------
def build_feature_table(dfs):
    app = dfs["application_train"].copy()
    app = fix_application_anomalies(app)

    # simple ratios in app
    for col in ["AMT_CREDIT", "AMT_ANNUITY", "AMT_INCOME_TOTAL"]:
        if col not in app.columns:
            raise ValueError(f"Missing expected column in application_train: {col}")

    app["CREDIT_INCOME_RATIO"] = safe_ratio(app["AMT_CREDIT"], app["AMT_INCOME_TOTAL"])
    app["ANNUITY_INCOME_RATIO"] = safe_ratio(app["AMT_ANNUITY"], app["AMT_INCOME_TOTAL"])
    app["CREDIT_TERM"] = safe_ratio(app["AMT_CREDIT"], app["AMT_ANNUITY"])

    # Aggregations
    bureau_agg = aggregate_bureau(dfs["bureau"])
    bb_agg = aggregate_bureau_balance(dfs["bureau_balance"], dfs["bureau"])
    prev_agg = aggregate_previous_application(dfs["previous_application"])
    inst_agg = aggregate_installments(dfs["installments_payments"])
    pos_agg = aggregate_pos_cash(dfs["POS_CASH_balance"])
    cc_agg = aggregate_credit_card(dfs["credit_card_balance"])

    # Merge into app
    out = app.merge(bureau_agg, on="SK_ID_CURR", how="left")
    out = out.merge(bb_agg, on="SK_ID_CURR", how="left")
    out = out.merge(prev_agg, on="SK_ID_CURR", how="left")
    out = out.merge(inst_agg, on="SK_ID_CURR", how="left")
    out = out.merge(pos_agg, on="SK_ID_CURR", how="left")
    out = out.merge(cc_agg, on="SK_ID_CURR", how="left")

    # Missing flags for strategic columns (value + missingness both can be predictive)
    strategic = [c for c in ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3",
                            "bureau_debt_sum", "bb_bad_ratio",
                            "prev_approval_rate", "inst_late_ratio",
                            "cc_util_mean", "pos_late_ratio"] if c in out.columns]
    out = add_missing_flags(out, strategic)

    return out

# -----------------------
# 5) CLEANING + ENCODING
# -----------------------
def clean_and_encode(full_df, target_col="TARGET", drop_cols=("SK_ID_CURR",), fill_cat="Unknown"):
    df = full_df.copy()

    # Identify columns
    cat_cols = df.select_dtypes(include="object").columns.tolist()
    num_cols = df.select_dtypes(exclude="object").columns.tolist()

    # Fill categorical
    for c in cat_cols:
        df[c] = df[c].fillna(fill_cat)

    # Fill numeric (median), excluding target
    if target_col in df.columns:
        num_cols_wo_target = [c for c in num_cols if c != target_col]
    else:
        num_cols_wo_target = num_cols

    medians = {}
    for c in num_cols_wo_target:
        if pd.api.types.is_numeric_dtype(df[c]):
            med = df[c].median()
            medians[c] = med
            df[c] = df[c].fillna(med)

    # Build X/y
    y = df[target_col].astype(int) if target_col in df.columns else None
    X = df.drop(columns=[target_col], errors="ignore")

    # drop IDs
    for c in drop_cols:
        if c in X.columns:
            X = X.drop(columns=[c])

    # One-hot encode
    X = pd.get_dummies(X, dummy_na=False)

    return X, y, df, medians

# -----------------------
# 6) RUN PIPELINE
# -----------------------
dfs = load_home_credit_data(DATA_PATH)

full_features = build_feature_table(dfs)
print("Merged feature table:", full_features.shape)

X, y, cleaned_df, medians = clean_and_encode(full_features)
print("X shape:", X.shape, "y shape:", y.shape)

# Sanity checks
print("NaN left in X:", int(X.isna().any().sum()))
print("Target rate:", y.mean())

# Optional: save
# cleaned_df.to_csv("home_credit_cleaned_with_aggs.csv", index=False)
# X.to_parquet("home_credit_X.parquet")


Merged feature table: (307511, 188)
X shape: (307511, 316) y shape: (307511,)
NaN left in X: 0
Target rate: 0.08072881945686496


# 2. Feature Engineering

# 2.a. Application Level

In [3]:
import numpy as np
import pandas as pd

def add_app_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # --- Basic ratios (some already in your pipeline, keep safe)
    if {"AMT_CREDIT","AMT_INCOME_TOTAL"}.issubset(df.columns):
        df["FE_CREDIT_INCOME_RATIO"] = df["AMT_CREDIT"] / df["AMT_INCOME_TOTAL"].replace(0, np.nan)

    if {"AMT_ANNUITY","AMT_INCOME_TOTAL"}.issubset(df.columns):
        df["FE_ANNUITY_INCOME_RATIO"] = df["AMT_ANNUITY"] / df["AMT_INCOME_TOTAL"].replace(0, np.nan)

    if {"AMT_CREDIT","AMT_ANNUITY"}.issubset(df.columns):
        df["FE_CREDIT_TERM"] = df["AMT_CREDIT"] / df["AMT_ANNUITY"].replace(0, np.nan)

    if {"AMT_GOODS_PRICE","AMT_CREDIT"}.issubset(df.columns):
        df["FE_GOODS_CREDIT_RATIO"] = df["AMT_GOODS_PRICE"] / df["AMT_CREDIT"].replace(0, np.nan)

    # --- Family / children normalization
    if {"CNT_FAM_MEMBERS","AMT_INCOME_TOTAL"}.issubset(df.columns):
        df["FE_INCOME_PER_FAMILY"] = df["AMT_INCOME_TOTAL"] / df["CNT_FAM_MEMBERS"].replace(0, np.nan)

    if {"CNT_CHILDREN","CNT_FAM_MEMBERS"}.issubset(df.columns):
        df["FE_CHILDREN_RATIO"] = df["CNT_CHILDREN"] / df["CNT_FAM_MEMBERS"].replace(0, np.nan)

    # --- Document / flag counts (these are weak individually, stronger as totals)
    doc_cols = [c for c in df.columns if c.startswith("FLAG_DOC")]
    if doc_cols:
        df["FE_FLAG_DOC_SUM"] = df[doc_cols].sum(axis=1)

    flag_cols = [c for c in df.columns if c.startswith("FLAG_") and c not in doc_cols]
    # Some FLAG_* are binary; sum can act as "how many flags"
    if flag_cols:
        # keep it bounded: ensure numeric
        df["FE_FLAG_SUM"] = df[flag_cols].select_dtypes(exclude="object").sum(axis=1)

    # --- External sources: mean / min / max / std + missing count
    ext = [c for c in ["EXT_SOURCE_1","EXT_SOURCE_2","EXT_SOURCE_3"] if c in df.columns]
    if ext:
        df["FE_EXT_MEAN"] = df[ext].mean(axis=1)
        df["FE_EXT_MIN"] = df[ext].min(axis=1)
        df["FE_EXT_MAX"] = df[ext].max(axis=1)
        df["FE_EXT_STD"] = df[ext].std(axis=1)
        df["FE_EXT_MISSING_COUNT"] = df[ext].isna().sum(axis=1)

    # --- Age / employment interactions (if you created AGE_YEARS / EMPLOYED_YEARS)
    if {"AGE_YEARS","EMPLOYED_YEARS"}.issubset(df.columns):
        df["FE_EMPLOYED_AGE_RATIO"] = df["EMPLOYED_YEARS"] / df["AGE_YEARS"].replace(0, np.nan)

    return df


# 2.b. Recent Behavior

In [6]:
def add_recent_installment_features(inst: pd.DataFrame) -> pd.DataFrame:
    """
    installments_payments: DAYS_INSTALMENT negative (past).
    We'll define "recent window" by DAYS_INSTALMENT >= -365 (last 12 months), >= -180 (last 6 months).
    Returns SK_ID_CURR-level features.
    """
    i = inst.copy()

    i["LATE"] = (i["DAYS_ENTRY_PAYMENT"] > i["DAYS_INSTALMENT"]).astype(int)
    i["DAYS_LATE"] = (i["DAYS_ENTRY_PAYMENT"] - i["DAYS_INSTALMENT"]).clip(lower=0)

    feats = []
    for window, name in [(365, "12M"), (180, "6M")]:
        recent = i[i["DAYS_INSTALMENT"] >= -window].copy()
        agg = recent.groupby("SK_ID_CURR").agg(
            **{
                f"FE_INST_LATE_RATIO_{name}": ("LATE", "mean"),
                f"FE_INST_DAYS_LATE_MEAN_{name}": ("DAYS_LATE", "mean"),
                f"FE_INST_COUNT_{name}": ("SK_ID_PREV", "count"),
            }
        ).reset_index()
        feats.append(agg)

    # merge the two windows together
    out = feats[0]
    out = out.merge(feats[1], on="SK_ID_CURR", how="outer")
    return out


# POS: SK-DPD

In [7]:
def add_recent_pos_features(pos: pd.DataFrame) -> pd.DataFrame:
    p = pos.copy()
    p["POS_LATE"] = (p["SK_DPD"] > 0).astype(int)

    feats = []
    for window, name in [(12, "12M"), (6, "6M")]:
        # MONTHS_BALANCE: 0 is current, -1 previous month, etc.
        recent = p[p["MONTHS_BALANCE"] >= -window].copy()
        agg = recent.groupby("SK_ID_CURR").agg(
            **{
                f"FE_POS_LATE_RATIO_{name}": ("POS_LATE", "mean"),
                f"FE_POS_SKDPD_MEAN_{name}": ("SK_DPD", "mean"),
                f"FE_POS_COUNT_{name}": ("SK_ID_PREV", "count"),
            }
        ).reset_index()
        feats.append(agg)

    out = feats[0].merge(feats[1], on="SK_ID_CURR", how="outer")
    return out


# Credit Card: utilization / balance

In [8]:
def add_recent_cc_features(cc: pd.DataFrame) -> pd.DataFrame:
    c = cc.copy()

    if {"AMT_BALANCE","AMT_CREDIT_LIMIT_ACTUAL"}.issubset(c.columns):
        c["CC_UTIL"] = c["AMT_BALANCE"] / c["AMT_CREDIT_LIMIT_ACTUAL"].replace(0, np.nan)
    else:
        c["CC_UTIL"] = np.nan

    feats = []
    for window, name in [(12, "12M"), (6, "6M")]:
        recent = c[c["MONTHS_BALANCE"] >= -window].copy()
        agg = recent.groupby("SK_ID_CURR").agg(
            **{
                f"FE_CC_UTIL_MEAN_{name}": ("CC_UTIL", "mean"),
                f"FE_CC_BAL_MEAN_{name}": ("AMT_BALANCE", "mean"),
                f"FE_CC_COUNT_{name}": ("SK_ID_PREV", "count"),
            }
        ).reset_index()
        feats.append(agg)

    out = feats[0].merge(feats[1], on="SK_ID_CURR", how="outer")
    return out


# 2.c. Group-based normalization

In [9]:
def add_group_zscores(df: pd.DataFrame, group_col: str, value_cols: list[str]) -> pd.DataFrame:
    df = df.copy()
    if group_col not in df.columns:
        return df

    for v in value_cols:
        if v not in df.columns:
            continue
        grp_mean = df.groupby(group_col)[v].transform("mean")
        grp_std = df.groupby(group_col)[v].transform("std")
        df[f"FE_{v}_Z_IN_{group_col}"] = (df[v] - grp_mean) / grp_std.replace(0, np.nan)
    return df


In [10]:
# --- 1) full_features 
full_features = build_feature_table(dfs)

# --- 2) app-level FE ---
full_features = add_app_features(full_features)

# --- 3) recent behavior FE from raw tables ---
inst_recent = add_recent_installment_features(dfs["installments_payments"])
pos_recent  = add_recent_pos_features(dfs["POS_CASH_balance"])
cc_recent   = add_recent_cc_features(dfs["credit_card_balance"])

full_features = full_features.merge(inst_recent, on="SK_ID_CURR", how="left")
full_features = full_features.merge(pos_recent,  on="SK_ID_CURR", how="left")
full_features = full_features.merge(cc_recent,   on="SK_ID_CURR", how="left")

# --- 4) group-based z-scores  ---
full_features = add_group_zscores(
    full_features,
    group_col="ORGANIZATION_TYPE",
    value_cols=[c for c in ["AMT_INCOME_TOTAL","AMT_CREDIT","AMT_ANNUITY"] if c in full_features.columns]
)

# --- 5) Add missing flags for the new engineered features too (keep signal)
engineered_cols = [c for c in full_features.columns if c.startswith("FE_")]
full_features = add_missing_flags(full_features, engineered_cols[:60])  # limit: too many flags can explode

# --- 6) Clean + encode into X/y ---
X, y, cleaned_df, medians = clean_and_encode(full_features)

print("Final shapes:", X.shape, y.shape)
print("NaN left in X:", int(X.isna().any().sum()))
print("Target rate:", y.mean())


Final shapes: (307511, 386) (307511,)
NaN left in X: 0
Target rate: 0.08072881945686496


# 3. Feature Selection

# 3.a. Hard filtering

In [13]:
def drop_useless_features(df, target_col="TARGET"):
    drop_cols = []

    # ID-like columns
    for c in df.columns:
        if c.startswith("SK_ID"):
            drop_cols.append(c)

    # Constant columns
    nunique = df.nunique()
    const_cols = nunique[nunique <= 1].index.tolist()
    drop_cols.extend(const_cols)

    # Explicitly known useless flags (domain-based)
    known_useless = [c for c in df.columns if c.startswith("FLAG_DOCUMENT")]
    drop_cols.extend(known_useless)

    drop_cols = list(set(drop_cols))
    df = df.drop(columns=drop_cols, errors="ignore")

    return df, drop_cols


In [14]:
fs_df = full_features.copy()
fs_df, dropped_stage1 = drop_useless_features(fs_df)

print("Dropped in stage 1:", len(dropped_stage1))


Dropped in stage 1: 27


# 3.b. Low variance

In [15]:
def drop_low_variance(df, threshold=0.99):
    drop_cols = []
    for c in df.columns:
        if df[c].dtype == "object":
            continue
        top_freq = df[c].value_counts(normalize=True, dropna=False).iloc[0]
        if top_freq >= threshold:
            drop_cols.append(c)

    df = df.drop(columns=drop_cols, errors="ignore")
    return df, drop_cols


In [16]:
fs_df, dropped_low_var = drop_low_variance(fs_df, threshold=0.99)
print("Dropped low-variance:", len(dropped_low_var))


Dropped low-variance: 12


# 3.b.1. redundant

In [17]:
def drop_highly_correlated(df, threshold=0.95):
    num_df = df.select_dtypes(exclude="object").copy()
    corr = num_df.corr().abs()

    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

    drop_cols = [
        column for column in upper.columns
        if any(upper[column] > threshold)
    ]

    df = df.drop(columns=drop_cols, errors="ignore")
    return df, drop_cols


In [18]:
fs_df, dropped_corr = drop_highly_correlated(fs_df, threshold=0.95)
print("Dropped correlated:", len(dropped_corr))


Dropped correlated: 62


# 3.c. Spearman correlation

In [19]:
def select_numeric_by_target_corr(df, target_col="TARGET", min_abs_corr=0.01):
    num_cols = df.select_dtypes(exclude="object").columns
    num_cols = [c for c in num_cols if c != target_col]

    corr = df[num_cols + [target_col]].corr(method="spearman")[target_col]
    corr = corr.drop(target_col)

    keep = corr[abs(corr) >= min_abs_corr].index.tolist()
    drop = corr[abs(corr) < min_abs_corr].index.tolist()

    return keep, drop, corr.sort_values()


In [21]:
keep_num, drop_num, num_corr = select_numeric_by_target_corr(
    fs_df, min_abs_corr=0.01
)

fs_df = fs_df.drop(columns=drop_num)
print("Dropped weak numeric:", len(drop_num))


Dropped weak numeric: 0


# 3.c.1. Default rate variance

In [22]:
def select_categorical_by_target_spread(df, target_col="TARGET", min_std=0.005):
    cat_cols = df.select_dtypes(include="object").columns

    keep, drop = [], []
    for c in cat_cols:
        rates = df.groupby(c)[target_col].mean()
        if rates.std() >= min_std:
            keep.append(c)
        else:
            drop.append(c)

    return keep, drop


In [23]:
keep_cat, drop_cat = select_categorical_by_target_spread(fs_df)

fs_df = fs_df.drop(columns=drop_cat)
print("Dropped weak categorical:", len(drop_cat))


Dropped weak categorical: 2


In [24]:
print("Remaining features:", fs_df.shape)


Remaining features: (307511, 131)


In [28]:
fs_df.to_csv("fs_df.csv", index=False)